In [57]:
import numpy as np
import torch, pickle
import lightgbm as lgb
from hummingbird import convert_sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer

In [69]:
# use some dataset from sklearn

X, y = load_breast_cancer(return_X_y=True)
nrows=15000
X = X[0:nrows]
y = y[0:nrows]
X_torch = torch.from_numpy(X).float()

In [70]:
# Create and train a model
model = RandomForestClassifier(n_estimators=10, max_depth=10)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [71]:
# Use hummingbird to convert your sklearn model to pytorch
pytorch_model = convert_sklearn(
    model, 
    extra_config = {"tree_implementation": "gemm"})

In [72]:
%%timeit -r 3

#time for skl
skl = model.predict(X)

1.53 ms ± 5.03 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


In [73]:
%%timeit -r 3

# time for hummingbird - CPU
pytorch_model.to('cpu')
hum_cpu = pytorch_model(X_torch.float())

2.35 ms ± 254 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [74]:
%%timeit -r 3

# time for hummingbird - GPU. Note that you must have a GPU-enabled machine.
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

432 µs ± 5.03 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


In [75]:
# make sure Hummingbird output matches Scikit-learn as expected
# (note that we have to recreate skl since `timeit` make it out of scope here)
skl = model.predict_proba(X)
pytorch_model.to('cuda')
hum_gpu = pytorch_model(X_torch.to('cuda'))

np.testing.assert_allclose(skl, hum_gpu[1].data.to('cpu').numpy(), rtol=1e-6, atol=1e-6)
